In [1]:
import pandas as pd
import glob
from tqdm import tqdm

# get all the json files in votes_out_2023_07_21
file_names = glob.glob('votes_out_2023_07_21/*.json')
len(file_names)

6782

In [2]:
# monkeypatch json
# monkeypatch using standard python json module

import json

pd.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# monkeypatch using faster simplejson module
import simplejson
pd.io.json._json.loads = lambda s, *a, **kw: simplejson.loads(s)

# normalising (unnesting) at the same time (for nested jsons)
pd.io.json._json.loads = lambda s, *a, **kw: pd.json_normalize(simplejson.loads(s))

In [3]:
# read them all as one big dataframe
df = pd.concat([pd.read_json(f) for f in tqdm(file_names)], ignore_index=True)
df.shape

  0%|          | 0/6782 [00:00<?, ?it/s]

  0%|          | 12/6782 [00:00<01:01, 110.94it/s]

  0%|          | 24/6782 [00:00<01:30, 74.29it/s] 

  1%|          | 34/6782 [00:00<01:27, 77.19it/s]

  1%|          | 43/6782 [00:00<01:37, 69.27it/s]

...

100%|█████████▉| 6761/6782 [01:11<00:00, 94.23it/s]

100%|█████████▉| 6771/6782 [01:11<00:00, 93.71it/s]

100%|█████████▉| 6781/6782 [01:12<00:00, 24.30it/s]

100%|██████████| 6782/6782 [01:12<00:00, 93.06it/s]

(5984663, 7)

In [4]:
df.head()

,id,voter,created,vp_by_strategy,vp,space.id,proposal.id
0,0xf6518f038e19abadfa7bcc1d01ff48f1919b18f4fe95...,0x1F4B988007C9C184c9760C31Eec3013d9657F405,1676304417,"[0, 0, 0, 0, 0, 0, 0, 1]",1.0,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...
1,0x61cab4b2f02eb464f28f4b2753a261c92a87a4551473...,0x93dad0F3Ce4F1bBce1d1E522AaCbd1664F0E1C0c,1676304364,"[0, 0, 1, 1, 1, 1, 1, 0]",5.0,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...
2,0xa2e4fb712f481436af39732fc0ce9c4e4a840ef8a229...,0x6f9b2e11FE2991eb68e11c2D90e4D74B7eE4aFA5,1676304183,"[0, 0, 0, 0, 0, 0, 0, 1]",1.0,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...
3,0x7c8d6a3406d637f0556caf600bd6484d03e57e4e0cd1...,0x4a334A59C4692AEB118c31eF1f2379fb10809089,1676304181,"[0, 0, 1, 1, 1, 1, 1, 0]",5.0,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...
4,0x774b39ba7a1e4b577dafcdd3632ff8e338ec6d3696f0...,0x7184CA841324a5A86A959df0c3c69502609FF2FE,1676304056,"[0, 0, 1, 1, 1, 1, 0, 1]",5.0,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...


In [5]:
# * platform
df['platform'] = 'snapshot'
# * rename space.id to platform_deployment_id
df.rename(columns={'space.id': 'platform_deployment_id'}, inplace=True)
# * rename proposal.id to proposal_id
df.rename(columns={'proposal.id': 'proposal_id'}, inplace=True)
# * rename id to vote_id
df.rename(columns={'id': 'vote_id'}, inplace=True)
# * voter -- the id of the person who made the vote, no chg
# * rename created to date -- this should be as a pandas date, not as an epoch
df.rename(columns={'created': 'date'}, inplace=True)
# * rename vp_by_strategy to choice -- this should be a string as this is not always true/false
df.rename(columns={'vp_by_strategy': 'choice'}, inplace=True)
# make string
df['choice'] = df['choice'].astype(str)
# * rename vp to weight -- the vote weight
df.rename(columns={'vp': 'weight'}, inplace=True)
# make int
df['weight'] = df['weight'].astype(int)
df.head()


/usr/lib/python3.11/site-packages/pandas/core/dtypes/astype.py:189: RuntimeWarning: invalid value encountered in cast
  return values.astype(dtype, copy=copy)


,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0xf6518f038e19abadfa7bcc1d01ff48f1919b18f4fe95...,0x1F4B988007C9C184c9760C31Eec3013d9657F405,1676304417,"[0, 0, 0, 0, 0, 0, 0, 1]",1,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
1,0x61cab4b2f02eb464f28f4b2753a261c92a87a4551473...,0x93dad0F3Ce4F1bBce1d1E522AaCbd1664F0E1C0c,1676304364,"[0, 0, 1, 1, 1, 1, 1, 0]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
2,0xa2e4fb712f481436af39732fc0ce9c4e4a840ef8a229...,0x6f9b2e11FE2991eb68e11c2D90e4D74B7eE4aFA5,1676304183,"[0, 0, 0, 0, 0, 0, 0, 1]",1,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
3,0x7c8d6a3406d637f0556caf600bd6484d03e57e4e0cd1...,0x4a334A59C4692AEB118c31eF1f2379fb10809089,1676304181,"[0, 0, 1, 1, 1, 1, 1, 0]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
4,0x774b39ba7a1e4b577dafcdd3632ff8e338ec6d3696f0...,0x7184CA841324a5A86A959df0c3c69502609FF2FE,1676304056,"[0, 0, 1, 1, 1, 1, 0, 1]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot


In [6]:
# make date a pandas date, parse epoch timestamp type 's'
df['date'] = pd.to_datetime(df['date'], unit='s')
df.head()

,vote_id,voter,date,choice,weight,platform_deployment_id,proposal_id,platform
0,0xf6518f038e19abadfa7bcc1d01ff48f1919b18f4fe95...,0x1F4B988007C9C184c9760C31Eec3013d9657F405,2023-02-13 16:06:57,"[0, 0, 0, 0, 0, 0, 0, 1]",1,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
1,0x61cab4b2f02eb464f28f4b2753a261c92a87a4551473...,0x93dad0F3Ce4F1bBce1d1E522AaCbd1664F0E1C0c,2023-02-13 16:06:04,"[0, 0, 1, 1, 1, 1, 1, 0]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
2,0xa2e4fb712f481436af39732fc0ce9c4e4a840ef8a229...,0x6f9b2e11FE2991eb68e11c2D90e4D74B7eE4aFA5,2023-02-13 16:03:03,"[0, 0, 0, 0, 0, 0, 0, 1]",1,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
3,0x7c8d6a3406d637f0556caf600bd6484d03e57e4e0cd1...,0x4a334A59C4692AEB118c31eF1f2379fb10809089,2023-02-13 16:03:01,"[0, 0, 1, 1, 1, 1, 1, 0]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot
4,0x774b39ba7a1e4b577dafcdd3632ff8e338ec6d3696f0...,0x7184CA841324a5A86A959df0c3c69502609FF2FE,2023-02-13 16:00:56,"[0, 0, 1, 1, 1, 1, 0, 1]",5,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,snapshot


In [7]:
df = df[['platform', 'platform_deployment_id', 'proposal_id', 'vote_id', 'voter', 'date', 'choice', 'weight']]

In [8]:
df.head()

,platform,platform_deployment_id,proposal_id,vote_id,voter,date,choice,weight
0,snapshot,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,0xf6518f038e19abadfa7bcc1d01ff48f1919b18f4fe95...,0x1F4B988007C9C184c9760C31Eec3013d9657F405,2023-02-13 16:06:57,"[0, 0, 0, 0, 0, 0, 0, 1]",1
1,snapshot,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,0x61cab4b2f02eb464f28f4b2753a261c92a87a4551473...,0x93dad0F3Ce4F1bBce1d1E522AaCbd1664F0E1C0c,2023-02-13 16:06:04,"[0, 0, 1, 1, 1, 1, 1, 0]",5
2,snapshot,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,0xa2e4fb712f481436af39732fc0ce9c4e4a840ef8a229...,0x6f9b2e11FE2991eb68e11c2D90e4D74B7eE4aFA5,2023-02-13 16:03:03,"[0, 0, 0, 0, 0, 0, 0, 1]",1
3,snapshot,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,0x7c8d6a3406d637f0556caf600bd6484d03e57e4e0cd1...,0x4a334A59C4692AEB118c31eF1f2379fb10809089,2023-02-13 16:03:01,"[0, 0, 1, 1, 1, 1, 1, 0]",5
4,snapshot,kuiclub.eth,0x94d08ed19ce671d909a0bb791be11dc8a9d9c2ff0a8c...,0x774b39ba7a1e4b577dafcdd3632ff8e338ec6d3696f0...,0x7184CA841324a5A86A959df0c3c69502609FF2FE,2023-02-13 16:00:56,"[0, 0, 1, 1, 1, 1, 0, 1]",5


In [9]:
# save
df.to_csv('snapshot_votes_small_proposals.csv', index=False)